<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Sample-Bipartite-Networks" data-toc-modified-id="Sample-Bipartite-Networks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sample Bipartite Networks</a></span><ul class="toc-item"><li><span><a href="#TCGA" data-toc-modified-id="TCGA-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TCGA</a></span><ul class="toc-item"><li><span><a href="#high-impact-mutations" data-toc-modified-id="high-impact-mutations-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>high impact mutations</a></span></li><li><span><a href="#high-+-moderate-impact-mutations" data-toc-modified-id="high-+-moderate-impact-mutations-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>high + moderate impact mutations</a></span></li><li><span><a href="#TCGA-filtered-to-subtypes-represented-in-Samstein" data-toc-modified-id="TCGA-filtered-to-subtypes-represented-in-Samstein-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>TCGA filtered to subtypes represented in Samstein</a></span></li></ul></li><li><span><a href="#Samstein" data-toc-modified-id="Samstein-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Samstein</a></span></li></ul></li></ul></div>

# Imports

In [1]:
import sys,os,re
import gzip,pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy.stats as stats
import statsmodels.api as sm
sys.path.insert(0,'..')
import bigbets

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
2021-12-18 12:48:17,671:bigbets.ddr_data_object:INFO:Creating DDR Object


querying 1-72...done.
Finished.
1 input query terms found no hit:
	['SHFM1']
querying 1-1...done.
Finished.
1 input query terms found dup hits:
	[('SHFM1', 10)]
querying 1-276...done.
Finished.
12 input query terms found no hit:
	['APITD1', 'BRE', 'C17orf70', 'C19orf40', 'C1orf86', 'H2AFX', 'NDNL2', 'SHFM1', 'STRA13', 'TCEB1', '
querying 1-12...done.
Finished.
5 input query terms found dup hits:
	[('APITD1', 2), ('BRE', 3), ('SHFM1', 10), ('STRA13', 2), ('TCEB3', 2)]


2021-12-18 12:48:24,382:bigbets.ddr_data_object:INFO:Loading Chromatin remodelling genes from GO


  EXISTS: go-basic.obo
  EXISTS: gene2go
HMS:0:00:04.623708 335,439 annotations READ: gene2go 
1 taxids stored: 9606
12453 IDs in association branch, BP
12453 GO terms associated with human NCBI Entrez GeneIDs
go-basic.obo: fmt(1.2) rel(2021-11-16) 47,155 GO Terms; optional_attrs(comment def relationship synonym xref)
querying 1-267...done.
Finished.


/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/biothings_client/base.py:129: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(obj)


In [2]:
def compile_all_runs(runs_dir):
    # load individual runs
    data_files=[ os.path.join(runs_dir,f) for f in os.listdir(runs_dir) if re.search("genes.pickle",f)]

    #we keep only keep track of the mean tmb for each gene and each sample across all runs
    combined_mean_tmbs={}
    data_files
#     genes2use=spec_by_genes_df.columns
    for i,file in enumerate(data_files):
        print(i)
        with gzip.open(file,'rb') as fh:
            cdata=pickle.load(fh)
        for ii,gene in enumerate(cdata.keys()):
            if ii%1000==0:
                print(ii,end=', ')

            all_x,all_ecdf=cdata.loc[gene]
            all_x=np.array(all_x)
            med_tmbs=np.mean(np.array(all_x),axis=1)
            combined_mean_tmbs[gene]=np.append(combined_mean_tmbs.get(gene,np.array([])),med_tmbs)
        del cdata
        print()
    return combined_mean_tmbs
    

In [16]:
def calculate_bigbet_scores_dataframe(combined_mean_tmbs,spec_by_genes_df,snv_indel_df):
    #means from network reshuffling
    means_rewiring_df=pd.DataFrame()

    genes2use=spec_by_genes_df.columns
    samples=spec_by_genes_df.index
    gene_sizes=np.sum(spec_by_genes_df,axis=0)


    for i,gene in enumerate(genes2use):
        if i%1000==0:
             print(i,end=', ')
         
        means_rewiring_df.loc[gene,'gene_mut_cnt']=gene_sizes[gene]

        try:
            med_tmbs=combined_mean_tmbs[gene]
        except KeyError:
            continue
        obs_tmb=np.mean(np.log1p(snv_indel_df.loc[samples[np.nonzero(spec_by_genes_df.loc[:,gene].values)[0]],'tmb'].dropna()))
        means_rewiring_df.loc[gene,'gene_mut_cnt']=gene_sizes[gene]
        cmn=np.mean(med_tmbs)
        cstd=np.std(med_tmbs)
        means_rewiring_df.loc[gene,'obs_tmb']=obs_tmb
        means_rewiring_df.loc[gene,'mean']=cmn
        means_rewiring_df.loc[gene,'std']=cstd
        means_rewiring_df.loc[gene,'upper']=cmn+1.96*cstd
        means_rewiring_df.loc[gene,'lower']=cmn-1.96*cstd
        med_tmbs = np.append(med_tmbs,obs_tmb)
        means_rewiring_df.loc[gene,'bigbets']=stats.mstats.zscore(a=med_tmbs)[-1]
        
        mwu,pval=bigbets.bipartite_helper_functions.cal_mwu_pval([gene],
                                                        spec_by_genes_df,snv_indel_df,
                                                        outgroup=spec_by_genes_df.index) #just use all samples outgroup for comaprison here.
        
        means_rewiring_df.loc[gene,'mwu_U']=mwu
        means_rewiring_df.loc[gene,'mwu_pval']=pval
        
    means_rewiring_df['path']=list(map(lambda x: bigbets.ddr_data_object.myddr_obj.gene_2_path_dict.get(x,['None'])[0],means_rewiring_df.index))
    
    
    means_rewiring_df['pval_bigbets'] = stats.norm.sf(abs(means_rewiring_df['bigbets'])) #one-sided
    pvals=means_rewiring_df['pval_bigbets']
    means_rewiring_df['padj_bigbets']=np.nan
    ind2keep=np.where(~np.isnan(pvals))[0]
    is_sig,padj_nona,_,_ = sm.stats.multipletests(alpha=.05,pvals=pvals[ind2keep],method='fdr_bh')
    means_rewiring_df.loc[means_rewiring_df.index[ind2keep],'padj_bigbets']=padj_nona
    means_rewiring_df.loc[means_rewiring_df.index[ind2keep],'bigbets_sig']=is_sig

    pvals=means_rewiring_df['mwu_pval']
    means_rewiring_df['mwu_padj']=np.nan
    ind2keep=np.where(~np.isnan(pvals))[0]
    is_sig,padj_nona,_,_ = sm.stats.multipletests(alpha=.05,pvals=pvals[ind2keep],method='fdr_bh')
    means_rewiring_df.loc[means_rewiring_df.index[ind2keep],'mwu_padj']=padj_nona
    means_rewiring_df.loc[means_rewiring_df.index[ind2keep],'mwu_sig']=is_sig

    means_rewiring_df['-log10_padj']=-1*np.log10(means_rewiring_df['mwu_padj'])
    
    return means_rewiring_df


# Sample Bipartite Networks

## TCGA

In [4]:
tcga_data=bigbets.TCGA_Data()
data_array=tcga_data.tcga_spec_by_all_genes

samples=data_array.index.values
features=data_array.columns.values
data_array=sparse.lil_matrix(data_array.values)

data_array_wmod=tcga_data.tcga_spec_by_all_genes_wmod
#overlap between two sets. 
data_array_wmod=data_array_wmod.loc[samples,features]

samples_wmod=data_array_wmod.index.values
features_wmod=data_array_wmod.columns.values
data_array_wmod=sparse.lil_matrix(data_array_wmod.values)

tcga_snv_indel_df=tcga_data.tcga_snv_indel_df
path_2_genes=bigbets.ddr_data_object.myddr_obj.path_2_genes_dict
gene_2_path_dict=bigbets.ddr_data_object.myddr_obj.gene_2_path_dict



2021-12-18 12:48:34,408:bigbets.load_tcga_dataset:INFO:Loading TCGA dataset
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
2021-12-18 12:49:45,141:bigbets.load_tcga_dataset:INFO:alteration_data_tcga.shape: (3600963, 25)
2021-12-18 12:50:03,752:bigbets.load_tcga_dataset:INFO:Filtering TCGA Dataset


tcga_alt_filt.shape (208682, 28)
querying 1-274...done.
Finished.
12 input query terms found no hit:
	['C1orf86', 'APITD1', 'TCEB3', 'BRE', 'SHFM1', 'TCEB1', 'H2AFX', 'NDNL2', 'TCEB2', 'C17orf70', 'STRA
querying 1-12...done.
Finished.
5 input query terms found dup hits:
	[('APITD1', 2), ('TCEB3', 2), ('BRE', 3), ('SHFM1', 10), ('STRA13', 2)]


### high impact mutations

In [14]:
bipartite_samples_dir=bigbets.file_locations.bipartite_samples_dir
high_impact_samples_dir=os.path.join(bipartite_samples_dir,'sampled_data/tcga_all_genes_high')
runs_300=os.path.join(high_impact_samples_dir,"smaller_runs_300")

In [ ]:
#Code below runs the bipartite network sampling.  Was executed on desktop with 8 cores running. 
#run parallel chains in smaller chunks and save output as we go. 
#define function

def get_tmb_dist_tcga(mat,samples,features):
    return bigbets.get_tmb_dist_both_path_and_genes(mat,samples,features,
                                                       tcga_snv_indel_df, path_2_genes)

tot_edges=np.sum(np.sum(data_array))
print(tot_edges)
samplesperrun=300


prefix='tgca_all_high_wpolyphen_{:d}_{:d}'
for i in range(20):
    print("on loop ",i)
    parallel_rewire=bigbets.ParallelRewiringChains(array=data_array,samples=samples,
                                                      features=features,
                                                      burninwires=300000,
                                                      nrewires_per_sample=300000,
                                                      func2call=get_tmb_dist_tcga,
                                                      nprocesses=8,nsamples=samplesperrun)
    parallel_rewire.run_allchains()
    data_path=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['paths'] for k,val in parallel_rewire.all_res.items()}))
    data_genes=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['genes'] for k,val in parallel_rewire.all_res.items()}))

    variables_2_save=['data_path','data_genes']

    if not os.path.exists(runs_300):
        os.makedirs(runs_300)
  
    for var in variables_2_save:
        outfile=os.path.join(runs_300,prefix.format(samplesperrun,i)+var+".pickle")
        with gzip.open(outfile,'wb') as fh:
            pickle.dump(globals()[var],fh)

    del parallel_rewire
    del data_genes
    del data_path

In [15]:
combined_med_tmb=compile_all_runs(runs_300)


0
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 
1
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 
2
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 
3
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 
4
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 
5
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 


In [73]:
means_rewiring_df=calculate_bigbet_scores_dataframe(combined_med_tmb,tcga_data.tcga_spec_by_all_genes,
                                                   tcga_data.tcga_snv_indel_df)

means_rewiring_df.to_csv(bigbets.file_locations.bigbet_scores_file)

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


A1BG       0.497018
A1CF       0.421165
A2M        0.487801
A2ML1      0.447747
A3GALT2    0.432418
             ...   
ZYG11A     0.441789
ZYG11B     0.436350
ZYX        0.473169
ZZEF1      0.491924
ZZZ3       0.467381
Name: padj_bigbets, Length: 18224, dtype: float64

In [53]:
_,means_rewiring_df['padj_bigbets'],_,_ = sm.stats.multipletests(alpha=.05,pvals=means_rewiring_df['pval_bigbets'],method='fdr_bh')
means_rewiring_df['padj_bigbets']

A1BG      NaN
A1CF      NaN
A2M       NaN
A2ML1     NaN
A3GALT2   NaN
           ..
ZYG11A    NaN
ZYG11B    NaN
ZYX       NaN
ZZEF1     NaN
ZZZ3      NaN
Name: padj_bigbets, Length: 18224, dtype: float64

In [48]:
sampled_sizes=np.sum(tcga_data.tcga_spec_by_all_genes,axis=0)
#check that the sizes are all the same here
# data_files=[ os.path.join(runs_300,f) for f in os.listdir(runs_300) if re.search("genes.pickle",f)]
# combined_sizes={}
# data_files
# for i,file in enumerate(data_files):
#     with gzip.open(file,'rb') as fh:
#         cdata=pickle.load(fh)
#     for ii,gene in enumerate(cdata.keys()):
#         if ii%1000==0:
#             print(ii,end=', ')
#         all_x,all_ecdf=cdata.loc[gene]
#         all_x=np.array(all_x)
#         combined_sizes[gene]=all_x.shape[1]
#     break

# np.mean(sampled_sizes.values==means_rewiring_df.loc[sampled_sizes.index,'gene_mut_cnt'].values)

1.0

In [ ]:
# compile and save data


### high + moderate impact mutations

In [17]:
bipartite_samples_dir=bigbets.file_locations.bipartite_samples_dir
mod_impact_samples_dir=os.path.join(bipartite_samples_dir,'sampled_data/tcga_all_genes_moderate_high')
mod_runs_300=os.path.join(mod_impact_samples_dir,"smaller_runs_300")
if not os.path.exists(mod_runs_300):
    os.makedirs(mod_runs_300)

In [ ]:
#Code below runs the bipartite network sampling.  Was executed on desktop with 8 cores running. 
#run parallel chains in smaller chunks and save output as we go. 
#define function

def get_tmb_dist_tcga(mat,samples,features):
    return bigbets.get_tmb_dist_both_path_and_genes(mat,samples,features,
                                                       tcga_snv_indel_df, path_2_genes)

tot_edges=np.sum(np.sum(data_array))
print(tot_edges)
samplesperrun=300


prefix='tgca_all_high_moderate_wpolyphen_{:d}_{:d}'
for i in range(2):
    print("on loop ",i)
    parallel_rewire=bigbets.ParallelRewiringChains(array=data_array_wmod,samples=samples_wmod,
                                                  features=features_wmod,
                                                  burninwires=3000000,
                                                  nrewires_per_sample=3000000,
                                                  func2call=get_tmb_dist_tcga,
                                                  nprocesses=4,nsamples=200)
    parallel_rewire.run_allchains()
    data_path=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['paths'] for k,val in parallel_rewire.all_res.items()}))
    data_genes=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['genes'] for k,val in parallel_rewire.all_res.items()}))

    variables_2_save=['data_path','data_genes']

    if not os.path.exists(runs_300):
        os.makedirs(runs_300)
  
    for var in variables_2_save:
        outfile=os.path.join(mod_runs_300,prefix.format(samplesperrun,i)+var+".pickle")
        with gzip.open(outfile,'wb') as fh:
            pickle.dump(globals()[var],fh)

    del parallel_rewire
    del data_genes
    del data_path

In [18]:
combined_med_tmb=compile_all_runs(mod_runs_300)


0
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 
1
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 


In [19]:
means_rewiring_df=calculate_bigbet_scores_dataframe(combined_med_tmb,tcga_data.tcga_spec_by_all_genes_wmod,
                                                   tcga_data.tcga_snv_indel_df)

means_rewiring_df.to_csv(bigbets.file_locations.bigbet_moderate_included_scores_file)

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
# combined pathway level individual runs
combined_path_data={}

# load individual runs
data_files_path=[ os.path.join(mod_runs_300,f) for f in os.listdir(mod_runs_300) if re.search("path.pickle",f)]

#we keep track of the mean tmb for each gene and each sample across all runs
combined_cdfs_tmbs={}
combined_mean_tmbs={}
# ind2keep=drop_leading_inds(200,8,20)
for i,file in enumerate(data_files_path):
    print(i)
    with gzip.open(file,'rb') as fh:
        cdata=pickle.load(fh)
    for path in cdata.keys():
        print(path,end=', ')
        all_x,all_ecdf=cdata.loc[path]
        cmean_tmbs=[np.mean(x) for x in all_x] 
        combined_mean_tmbs[path]=combined_mean_tmbs.get(path,[])+cmean_tmbs
        if i==0:
            combined_cdfs_tmbs[path]=(np.array(all_x),np.array(all_ecdf))
        else:
            cur_comb_all_x,cur_comb_ecdf=combined_cdfs_tmbs[path]
            combined_cdfs_tmbs[path]=(np.concatenate([cur_comb_all_x,all_x],axis=0),
                                      np.concatenate([cur_comb_ecdf,np.array(all_ecdf)]))
    del cdata
    print()
    
all_ind=tcga_data.tcga_spec_by_all_genes_wmod.index
array2use=tcga_data.tcga_spec_by_all_genes_wmod
snv_df=tcga_data.tcga_snv_indel_df


ddr_path_bigbets=pd.DataFrame()
for path in combined_mean_tmbs.keys():
    mean_tmbs=combined_mean_tmbs[path]
    genes=bigbets.ddr_data_object.myddr_obj.path_2_genes_dict[path]
    ctmbs=bigbets.bipartite_helper_functions.get_tmb_values_genes(genes,array2use,snv_indel_df=snv_df)
    ctmbs=ctmbs.to_frame(name='tmb')
    ctmbs['sample']=ctmbs.index
    ctmbs['path']=path
    ctmbs.index=np.arange(ctmbs.shape[0])
#     path_tmb_vals=pd.concat([path_tmb_vals,ctmbs])
    ddr_path_bigbets.loc[path,['MWU_stat','MWU_pval']]=bigbets.bipartite_helper_functions.cal_mwu_pval(genes,array2use,
                                                              outgroup=all_ind,
                                                              snv_indel_df=snv_df)
    obs_mean_tmb=np.mean(np.log1p(ctmbs['tmb']))
    zscore=stats.mstats.zscore(np.append(mean_tmbs,obs_mean_tmb))[-1]
    ddr_path_bigbets.loc[path,'bigbets']=zscore
    ddr_path_bigbets.loc[path,'path']=path

ddr_path_bigbets['-1og10_MWU']=-1*np.log10(ddr_path_bigbets['MWU_pval'])
ddr_path_bigbets['pval_bigbets']=stats.norm.sf(abs(ddr_path_bigbets['bigbets'])) #one-sided
_,ddr_path_bigbets['padj_tcga'],_,_ = sm.stats.multipletests(alpha=.05,pvals=ddr_path_bigbets['pval_bigbets'],method='fdr_bh')

ddr_path_bigbets.to_csv(os.path.join(bigbets.file_locations.bipartite_samples_dir,'tcga_allgenes_moderate_high_path_rewiring_zscores.csv'))


0
BER, NER, MMR, FA, HR, NHEJ, DS, histone_modification_pathway, chromatin_remodel, 
1
BER, NER, MMR, FA, HR, NHEJ, DS, histone_modification_pathway, chromatin_remodel, 


### TCGA filtered to subtypes represented in Samstein

In [11]:
#TCGA filtered down to samstein cancer types 
tcga_data=bigbets.TCGA_Data()
tcga_data.get_cancer_types_filtered()

data_array=tcga_data.spec_by_genes_type_filt
samples=data_array.index.values
features=data_array.columns.values
data_array=sparse.lil_matrix(data_array.values)

tcga_snv_indel_df=tcga_data.tcga_snv_indel_df
path_2_genes=bigbets.ddr_data_object.myddr_obj.path_2_genes_dict
gene_2_path_dict=bigbets.ddr_data_object.myddr_obj.gene_2_path_dict



2021-12-18 13:24:53,147:bigbets.load_tcga_dataset:INFO:Loading TCGA Clinical data
../bigbets/load_clinical_datasets.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddr_bigbets_filt_df.sort_values(by='bigbets', inplace=True)
2021-12-18 13:26:10,781:root:INFO:BER 56
2021-12-18 13:26:10,855:root:INFO:NER 112
2021-12-18 13:26:10,930:root:INFO:MMR 204
2021-12-18 13:26:11,004:root:INFO:FA 141
2021-12-18 13:26:11,081:root:INFO:HR 402
2021-12-18 13:26:11,155:root:INFO:NHEJ 175
2021-12-18 13:26:11,233:root:INFO:DS 305
2021-12-18 13:26:11,369:root:INFO:histone_modification_pathway 2381
2021-12-18 13:26:11,536:root:INFO:chromatin_remodel 2879


In [10]:
bipartite_samples_dir=bigbets.file_locations.bipartite_samples_dir
samstein_filt_tcga=os.path.join(bipartite_samples_dir,'sampled_data/tcga_samstein_cancer_types')
samfilt_runs=os.path.join(samstein_filt_tcga,"smaller_runs_300")

In [81]:
#Code below runs the bipartite network sampling.  Was executed on desktop with 8 cores running. 
#run parallel chains in smaller chunks and save output as we go. 
#define function

def get_tmb_dist_tcga(mat,samples,features):
    return bigbets.get_tmb_dist_both_path_and_genes(mat,samples,features,
                                                     tcga_snv_indel_df, path_2_genes)

tot_edges=np.sum(np.sum(data_array))
print(tot_edges)
samplesperrun=300


prefix='tgca_samstein_cancer_types_high_wpolyphen_{:d}_{:d}'
for i in range(4):
    print("on loop ",i)
    parallel_rewire=bigbets.ParallelRewiringChains(array=data_array,samples=samples,
                                                      features=features,
                                                      burninwires=300000,
                                                      nrewires_per_sample=300000,
                                                      func2call=get_tmb_dist_tcga,
                                                      nprocesses=8,nsamples=samplesperrun)
    parallel_rewire.run_allchains()
    data_path=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['paths'] for k,val in parallel_rewire.all_res.items()}))
    data_genes=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['genes'] for k,val in parallel_rewire.all_res.items()}))

    variables_2_save=['data_path','data_genes']

    if not os.path.exists(samfilt_runs):
        os.makedirs(samfilt_runs)
  
    for var in variables_2_save:
        outfile=os.path.join(samfilt_runs,prefix.format(samplesperrun,i)+var+".pickle")
        with gzip.open(outfile,'wb') as fh:
            pickle.dump(globals()[var],fh)

    del parallel_rewire
    del data_genes
    del data_path

(4779, 18224)

In [11]:
combined_med_tmb=compile_all_runs(samfilt_runs)
means_rewiring_df=calculate_bigbet_scores_dataframe(combined_med_tmb,tcga_data.spec_by_genes_type_filt,
                                                   tcga_data.tcga_snv_indel_df)
means_rewiring_df.to_csv(os.path.join(bigbets.file_locations.bipartite_samples_dir,
                                      'tcga_filter_samstein_cancertypes_rewiring_zscores.csv'))

0
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 
1
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 
2
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 
3
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 
0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 

In [14]:
# combined pathway level individual runs
combined_path_data={}

# load individual runs
data_files_path=[ os.path.join(samfilt_runs,f) for f in os.listdir(samfilt_runs) if re.search("path.pickle",f)]

#we keep track of the mean tmb for each gene and each sample across all runs
combined_cdfs_tmbs={}
combined_mean_tmbs={}
# ind2keep=drop_leading_inds(200,8,20)
for i,file in enumerate(data_files_path):
    print(i)
    with gzip.open(file,'rb') as fh:
        cdata=pickle.load(fh)
    for path in cdata.keys():
        print(path,end=', ')
        all_x,all_ecdf=cdata.loc[path]
        cmean_tmbs=[np.mean(x) for x in all_x] 
        combined_mean_tmbs[path]=combined_mean_tmbs.get(path,[])+cmean_tmbs
        if i==0:
            combined_cdfs_tmbs[path]=(np.array(all_x),np.array(all_ecdf))
        else:
            cur_comb_all_x,cur_comb_ecdf=combined_cdfs_tmbs[path]
            combined_cdfs_tmbs[path]=(np.concatenate([cur_comb_all_x,all_x],axis=0),
                                      np.concatenate([cur_comb_ecdf,np.array(all_ecdf)]))
    del cdata
    print()
    
all_ind=tcga_data.spec_by_genes_type_filt.index
array2use=tcga_data.spec_by_genes_type_filt
snv_df=tcga_data.tcga_snv_indel_df


ddr_path_bigbets=pd.DataFrame()
for path in combined_mean_tmbs.keys():
    mean_tmbs=combined_mean_tmbs[path]
    genes=bigbets.ddr_data_object.myddr_obj.path_2_genes_dict[path]
    ctmbs=bigbets.bipartite_helper_functions.get_tmb_values_genes(genes,array2use,snv_indel_df=snv_df)
    ctmbs=ctmbs.to_frame(name='tmb')
    ctmbs['sample']=ctmbs.index
    ctmbs['path']=path
    ctmbs.index=np.arange(ctmbs.shape[0])
#     path_tmb_vals=pd.concat([path_tmb_vals,ctmbs])
    ddr_path_bigbets.loc[path,['MWU_stat','MWU_pval']]=bigbets.bipartite_helper_functions.cal_mwu_pval(genes,array2use,
                                                              outgroup=all_ind,
                                                              snv_indel_df=snv_df)
    obs_mean_tmb=np.mean(np.log1p(ctmbs['tmb']))
    zscore=stats.mstats.zscore(np.append(mean_tmbs,obs_mean_tmb))[-1]
    ddr_path_bigbets.loc[path,'bigbets']=zscore
    ddr_path_bigbets.loc[path,'path']=path

ddr_path_bigbets['-1og10_MWU']=-1*np.log10(ddr_path_bigbets['MWU_pval'])
ddr_path_bigbets['pval_bigbets']=stats.norm.sf(abs(ddr_path_bigbets['bigbets'])) #one-sided
_,ddr_path_bigbets['padj_tcga'],_,_ = sm.stats.multipletests(alpha=.05,pvals=ddr_path_bigbets['pval_bigbets'],method='fdr_bh')

ddr_path_bigbets.to_csv(os.path.join(bigbets.file_locations.bipartite_samples_dir,'tcga_cancer_type_filt_path_rewiring_zscores.csv'))


0
BER, NER, MMR, FA, HR, NHEJ, DS, histone_modification_pathway, chromatin_remodel, 
1
BER, NER, MMR, FA, HR, NHEJ, DS, histone_modification_pathway, chromatin_remodel, 
2
BER, NER, MMR, FA, HR, NHEJ, DS, histone_modification_pathway, chromatin_remodel, 
3
BER, NER, MMR, FA, HR, NHEJ, DS, histone_modification_pathway, chromatin_remodel, 


## Samstein 

In [4]:
samstein=bigbets.SamsteinData()

bipartite_samples_dir=bigbets.file_locations.bipartite_samples_dir
samstein_samples_dir=os.path.join(bipartite_samples_dir,'sampled_data/samstein')
samstein_runs=os.path.join(samstein_samples_dir,"smaller_runs_300")
if not os.path.exists(samstein_runs):
    os.makedirs(samstein_runs)

2021-12-08 09:28:29,779:root:INFO:Loading Samstein clinical data
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (47,85) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


querying 1-469...done.
Finished.
29 input query terms found no hit:
	['PARK2', 'HIST1H1C', 'FAM46C', 'HIST1H3B', 'HIST1H3A', 'FAM175A', 'RFWD2', 'PAK7', 'WHSC1L1', 'WHSC
querying 1-29...done.


2021-12-08 09:28:34,328:root:INFO:Loading Samstein clinical data


Finished.
3 input query terms found dup hits:
	[('PARK2', 4), ('WHSC1L1', 2), ('HIST1H2BD', 2)]
2 input query terms found no hit:
	['RP11-145E5.5', 'RP11-211G3.3']


../bigbets/load_clinical_datasets.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddr_bigbets_filt_df.sort_values(by='bigbets', inplace=True)
2021-12-08 09:28:39,747:root:INFO:BER 27
2021-12-08 09:28:39,753:root:INFO:NER 156
2021-12-08 09:28:39,760:root:INFO:MMR 141
2021-12-08 09:28:39,766:root:INFO:FA 49
2021-12-08 09:28:39,773:root:INFO:HR 277
2021-12-08 09:28:39,780:root:INFO:NHEJ 0
2021-12-08 09:28:39,787:root:INFO:DS 223
2021-12-08 09:28:39,795:root:INFO:histone_modification_pathway 786
2021-12-08 09:28:39,805:root:INFO:chromatin_remodel 815


In [6]:
data_array=samstein.spec_by_genes
samples=data_array.index.values
features=data_array.columns.values
data_array=sparse.lil_matrix(data_array.values)
samstein_tmb_df=samstein.clinical_data.loc[:,['TMB']].rename({"TMB":'tmb'},axis=1)
path_2_genes=bigbets.ddr_data_object.myddr_obj.path_2_genes_dict
# samstein_tmb_df['tmb']=samstein_tmb_df['TMB']

In [ ]:
#Code below runs the bipartite network sampling.  Was executed on desktop with 8 cores running. 
#run parallel chains in smaller chunks and save output as we go. 
#define function

def get_tmb_dist_samstein(mat,samples,features):
    return bigbets.get_tmb_dist_both_path_and_genes(mat,samples,features,
                                                       samstein_tmb_df, path_2_genes)

tot_edges=np.sum(np.sum(data_array))
print(tot_edges)
samplesperrun=300


prefix='samstein_all_{:d}_{:d}'
for i in range(10):
    print("on loop ",i)
    parallel_rewire=bigbets.ParallelRewiringChains(array=data_array,samples=samples,
                                                      features=features,
                                                      burninwires=50000,
                                                      nrewires_per_sample=50000,
                                                      func2call=get_tmb_dist_samstein,
                                                      nprocesses=8,nsamples=samplesperrun)
    parallel_rewire.run_allchains()
    data_path=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['paths'] for k,val in parallel_rewire.all_res.items()}))
    data_genes=bigbets.bipartite_helper_functions.compile_all_runs(pd.DataFrame({ k: val['genes'] for k,val in parallel_rewire.all_res.items()}))

    variables_2_save=['data_path','data_genes']

    if not os.path.exists(samstein_runs):
        os.makedirs(samstein_runs)
  
    for var in variables_2_save:
        outfile=os.path.join(samstein_runs,prefix.format(samplesperrun,i)+var+".pickle")
        with gzip.open(outfile,'wb') as fh:
            pickle.dump(globals()[var],fh)

    del parallel_rewire
    del data_genes
    del data_path

In [10]:
combined_med_tmb=compile_all_runs(samstein_runs)
means_rewiring_df=calculate_bigbet_scores_dataframe(combined_med_tmb,samstein.spec_by_genes,
                                                   samstein_tmb_df)

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 

In [10]:
    means_rewiring_df['pval_bigbets'] = stats.norm.sf(abs(means_rewiring_df['bigbets'])) #one-sided
    _,means_rewiring_df['padj_bigbets'],_,_ = sm.stats.multipletests(alpha=.05,pvals=means_rewiring_df['pval_bigbets'],method='fdr_bh')

,gene_mut_cnt,obs_tmb,mean,std,upper,lower,bigbets,mwu_U,mwu_pval,path,pval_bigbets,padj_bigbets,mwu_sig,mwu_padj,-log10_padj
ABL1,29.0,3.625677,3.257120,0.185383,3.620470,2.893769,1.984813,41045.0,5.401603e-13,None,0.023583,0.117411,True,2.217500e-12,11.654136
ABRAXAS1,10.0,3.258174,3.313410,0.329290,3.958818,2.668001,-0.167685,13063.0,2.694236e-04,None,0.433416,0.490397,True,3.719477e-04,3.429518
ACVR1,17.0,3.533956,3.289151,0.243071,3.765570,2.812731,1.006459,22822.0,7.037960e-07,None,0.157097,0.326763,True,1.401602e-06,5.853375
AGO2,9.0,3.063537,3.306596,0.339075,3.971183,2.642008,-0.716467,11758.5,5.092791e-04,None,0.236852,0.390305,True,6.790388e-04,3.168105
AKT1,23.0,3.224658,3.281819,0.209348,3.692140,2.871498,-0.272944,29741.0,1.778459e-07,None,0.392448,0.473365,True,3.783267e-07,6.422133
